### Synopsis
This code averages and reduces g2 for temperature ramping (up and down) of protein H06

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

# mount_point = '/gdata/s8id-dmdtn/'
# fpath1 = mount_point + '2021-2/babnigg202107_2/cluster_results_QZ'
# fpath2 = mount_point + '2022-1/babnigg202203/cluster_results_QZ'

mount_point = "/home/8-id-i/2024-3/2024_1228_qz_llps_analysis/2025_08_analysis/"
fpath1 = mount_point + '2021-2/cluster_results_all'
fpath2 = mount_point + '2022-2/cluster_results_all'

coef_sam = 6.93e4  # Absolute scattering cross-section coefficient for the samples
coef_buf = 7.62e4  # Absolute scattering cross-section coefficient for the buffer


### Averaging of SA-XPCS results, temperature ramp up

In [2]:

avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='E0110', 
                                              prefix=fpath2,
                                              zone_idx='auto',
                                              num_cores=24)


index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    20.01	    21.28	  20.6458	 120
n=01	     21.3	    22.54	  21.9169	 120
n=02	    22.55	    23.81	  23.1837	 120
n=03	    23.82	    25.07	  24.4436	 120
n=04	    25.08	    26.35	  25.7158	 120
n=05	    26.36	    27.65	  27.0078	 120
n=06	    27.67	    28.97	  28.3151	 120
n=07	    28.98	    30.26	  29.6112	 120
n=08	    30.27	    31.58	  30.9063	 119
n=09	    31.59	    32.84	  32.2094	 119


### Averaging of SA-XPCS results, temperature ramp up

In [3]:

avg_ramp_down, _, _, _ = process_group(group='E0111', 
                                 prefix=fpath2,
                                 zone_idx='auto',
                                 num_cores=24)


index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    31.51	    32.83	  32.1641	 120
n=01	    30.22	     31.5	  30.8548	 120
n=02	    28.95	    30.21	  29.5736	 120
n=03	    27.64	    28.94	   28.298	 120
n=04	    26.37	    27.62	  26.9898	 120
n=05	    25.08	    26.36	  25.7127	 120
n=06	    23.77	    25.06	  24.4083	 120
n=07	    22.47	    23.76	  23.1137	 119
n=08	    21.19	    22.46	  21.8355	 119
n=09	    19.99	    21.19	  20.5543	 119


### Averaging of background

In [4]:

avg_bg, _, _, _ = process_group(group='D0138',
                                 num_sections=1,
                                 prefix=fpath2,
                                 zone_idx='auto',
                                 num_cores=24)


index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	     5.99	     6.01	   5.9998	 50


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [5]:

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = coef_sam*avg_ramp_up[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = coef_sam*avg_ramp_down[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
    